In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from timm.loss import SoftTargetCrossEntropy
import numpy as np
import math
import nltk
import mmseg
import inflect
import pickle
import json
# nltk.download('wordnet')

In [4]:
! pip install inflect

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/

In [8]:
from mmseg.datasets.ade import ADE20KDataset
from mmseg.datasets.bdd100k import BDD100KDataset
from mmseg.datasets.cityscapes import CityscapesDataset
from mmseg.datasets.coco_stuff import COCOStuffDataset
from mmseg.datasets.mapillary import MapillaryDataset_v1, MapillaryDataset_v2
from mmseg.datasets.pascal_context import PascalContextDataset, PascalContextDataset59
from mmseg.datasets.potsdam import PotsdamDataset
from mmseg.datasets.voc import PascalVOCDataset
remove_labs = ['background', 'unlabeled']
def to_lower_single(cls_labels):
    p=inflect.engine()
    cls_labels_new=[]
    for i,lab in enumerate(cls_labels):
        lab=lab.lower().strip()
        if p.singular_noun(lab):
            lab=p.singular_noun(lab)
        cls_labels_new.append(lab)
    cls_labels=list(set(cls_labels_new))
    return cls_labels



def save_list_to_file(lst, filename):
    # 将列表存储为文件
    with open(filename, 'wb') as f:
        pickle.dump(lst, f)

def load_list_from_file(filename):
    # 从文件中加载列表
    with open(filename, 'rb') as f:
        loaded_list = pickle.load(f)
    return loaded_list

all_labels = ()
all_labels += ADE20KDataset.METAINFO['classes']
all_labels += BDD100KDataset.METAINFO['classes']
all_labels += CityscapesDataset.METAINFO['classes']
all_labels += COCOStuffDataset.METAINFO['classes']
all_labels += MapillaryDataset_v1.METAINFO['classes']
all_labels += MapillaryDataset_v2.METAINFO['classes']
all_labels += PascalContextDataset.METAINFO['classes']
all_labels += PascalContextDataset59.METAINFO['classes']
all_labels += PotsdamDataset.METAINFO['classes']
all_labels += PascalVOCDataset.METAINFO['classes']
all_labels = to_lower_single(all_labels)
all_labels = list(set(all_labels)-set(remove_labs))
# save_list_to_file(all_labels, r'D:\BaiduSyncdisk\4.ResarchWorks\code\other\mmseg_labels.pkl')
print(all_labels)
print(len(all_labels))



# 将列表保存为JSON文件
filename = 'mmseg_labels_synonyms.json'
with open(filename, 'w') as file:
    json.dump(all_labels, file)

print("列表已保存为文件:", filename)


['washer', 'tunnel', 'lane marking - straight line', 'traffic light - general (single)', 'fence', 'traffic light', 'stage', 'ceiling', 'furniture-other', 'mountain', 'road', 'apparel', 'grandstand', 'waterfall', 'spoon', 'stool', 'couch', 'lane marking - arrow (straight)', 'traffic sign - ambiguou', 'lane marking - ambiguou', 'lane marking - stop line', 'giraffe', 'ceiling-other', 'manhole', 'plant-other', 'traffic light - pedestrian', 'bottle', 'plate', 'impervious_surface', 'conveyer belt', 'orange', 'zebra', 'service lane', 'coffee table', 'other rider', 'counter', 'cage', 'billboard', 'ambiguous barrier', 'tv', 'horse', 'fountain', 'lane marking - arrow (split right or straight)', 'boat', 'painting', 'water valve', 'floor-other', 'road shoulder', 'sky', 'lane marking - arrow (split left or straight)', 'track', 'stairway', 'tray', 'lane marking - text', 'wine glas', 'earth', 'bear', 'water', 'base', 'countertop', 'building', 'traffic sign frame', 'snowboard', 'bicyclist', 'parking a

In [6]:
def load_text_file(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    return text

file_path = 'mmseg_labels_synonyms.txt'
text = load_text_file(file_path)

# 去掉每行开头的 '-' 并分割成键值对
lines = text.strip().split('\n')
dictionary = {}

for i, line in enumerate(lines):
    key, value = line.strip().split(':')
    key = key.strip().lstrip('-').strip()
    value = value.strip()
    if key in dictionary.keys():
        print(key)
    dictionary[key] = [val.strip() for val in value.split(',')]

print(dictionary)
print(len(dictionary))


# 假设您已经生成了字典 dictionary

# 将字典保存为文件
filename = 'mmseg_labels_synonyms_dictionary.json'
with open(filename, 'w') as file:
    json.dump(dictionary, file)

print("字典已保存为文件:", filename)
value_list = [dictionary[key] for key in dictionary.keys()]
value = [lab for sub_list in value_list for lab in sub_list ]
keys = list(dictionary.keys())
all_cls = value+keys
print(all_cls)
print(len(all_cls),len(set(all_cls)))

bed
{'aeroplane': ['airplane', 'aircraft', 'jet', 'airliner', 'plane'], 'backpack': ['rucksack', 'knapsack', 'haversack', 'pack', 'sack'], 'furniture-other': ['furnishings', 'movables', 'fittings', 'appointments', 'fixtures'], 'traffic sign (front)': ['road sign', 'street sign', 'traffic signal'], 'wall-tile': ['ceramic tile', 'wall cladding', 'tilework'], 'ottoman': ['footstool', 'pouffe', 'stool', 'hassock'], 'clutter': ['mess', 'jumble', 'litter', 'muddle', 'chaos'], 'motorbike': ['motorcycle', 'bike', 'cycle', 'two-wheeler'], 'ashcan': ['trash can', 'garbage can', 'dustbin', 'waste bin'], 'bush': ['shrub', 'plant', 'hedge'], 'static': ['stationary', 'still', 'motionless'], 'carrot': ['root vegetable'], 'column': ['pillar', 'post', 'shaft'], 'waterdrop': ['droplet'], 'pillow': ['cushion'], 'floor': ['ground', 'flooring'], 'sheep': ['ewe (female sheep)', 'ram (male sheep)', 'lamb (young sheep)'], 'bookcase': ['bookshelf', 'bookrack'], 'lane marking (only) - other': ['road marking'], 

In [7]:
data = [['airplane', 'aircraft', 'jet', 'airliner', 'plane'],
        ['rucksack', 'knapsack', 'haversack', 'pack', 'sack']]

labels = []

for sublist in data:
    for i in range(len(sublist)):
        for j in range(len(sublist)):
            labels.append(1)  # 正例对的标签为1
        labels.append(0)  # 非正例对的标签为0

print(labels, len(labels))
print(len(labels))

[1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0] 60
60
